### Dialogue Chain Example

Chain is a pattern for sequentially engaging each agent in a dialogue by carrying the context of the conversation forward. This is useful for tasks that require multiple steps or when you want to maintain a coherent dialogue across different agents.

```mermaid
sequenceDiagram
    User ->> Sara: chat.send
    Sara ->> Diana: chat.reply
    Diana ->> User: chat.reply
```

The following code simulate an agent by using mock response.

In [1]:
import os

os.environ["LOG_LEVEL"] = "WARNING"
import json
import asyncio
from gai.sessions import SessionManager
from rich.console import Console

console = Console(force_terminal=True)
from gai.messages.typing import MessagePydantic
from gai.sessions.operations.chat import ChatResponder

# Helper function to create a node that returns mock agent responses


async def create_node(node_name: str, session_mgr: SessionManager, plan):
    async def input_chunks_handler(pydantic: MessagePydantic):
        if pydantic.body.type == "chat.reply":
            # Should never handle reply messages.
            raise ValueError("input_chunks_callback should not process reply messages.")

        # Return a simulated streamer
        with open(os.path.join("data", "mock_chat_response.json"), "r") as f:
            jsoned = json.loads(f.read())

        chunks = jsoned[node_name]["chunks"]
        for chunk in chunks:
            await asyncio.sleep(0.1)  # Simulate a delay for each chunk
            yield chunk

    async def completed_content_handler(pydantic: MessagePydantic):
        session_mgr.log_message(pydantic)

    node = ChatResponder(node_name=node_name, session_mgr=session_mgr)
    await node.subscribe(
        input_chunks_callback=input_chunks_handler,
        completed_content_callback=completed_content_handler,
    )
    node.plans[plan.dialogue_id] = plan.model_copy()
    return node


# Helper function for displaying the results


def print_title(title):
    console.print(
        f"""\n[bold bright_yellow]{len(title) * "="}\n{title}\n{len(title) * "="}[/bold bright_yellow]\n"""
    )


def show_messages(dialogue):
    print_title("dialogue.messages")
    print("The output below shows the messages saved in dialogue.messages:")
    for message in dialogue.messages:
        print(f"ID: {message.id}")
        print(f"  Sender: {message.header.sender}")
        print(f"  Recipient: {message.header.recipient}")
        print(f"Body:\n{json.dumps(message.body.model_dump(), indent=4)}")
        print("\n" + "-" * 10 + "\n")


---

### 1. Implementation with Local Dialogue

Chain within same process

In [2]:
import os
import asyncio
import json
from typing import Union
from gai.sessions.operations.chat import ChatResponder, ChatSender
from rich.console import Console

console = Console(force_terminal=True)

# Get the current directory

here = os.getcwd()

# Initialize the dialogue
from gai.lib.tests import make_local_tmp
from gai.sessions import SessionManager

app_dir = make_local_tmp()
session_mgr = SessionManager(file_path=os.path.join(app_dir, "dialogue.json"))
await session_mgr.start()

## Create plan

from gai.sessions.operations.handshake import HandshakeSender

plan = HandshakeSender.create_plan("""
    User ->> Sara
    Sara ->> Diana
    """)

## Register Sara and wait for User

sara = await create_node("Sara", session_mgr, plan)

## Register Diana and wait for User

diana = await create_node("Diana", session_mgr, plan)

# Register User and initiate session_mgr


async def output_chunks_handler(pydantic: MessagePydantic):
    if pydantic.body.chunk_no == 0:
        console.print(f"[bright_green]{pydantic.header.sender}[/bright_green] :")
        print(pydantic.body.chunk, end="", flush=True)
    else:
        if pydantic.body.chunk != "<eom>":
            print(pydantic.body.chunk, end="", flush=True)
        else:
            print("\n")


user = ChatSender(node_name="User", session_mgr=session_mgr)
await user.subscribe(output_chunks_callback=output_chunks_handler)

# Send message to Sara

step = await user.chat_send(user_message="Tell me a one paragraph story.", plan=plan)

while step:
    step = await user.next()

# Inspect the session_mgr

show_messages(session_mgr)

await session_mgr.stop()


Sara :

As she stood alone on the deserted beach, Emily felt an icy breeze whisper her name just as the clock struck midnight, exactly 20 years after her grandfather's mysterious disappearance.



Diana :

Firstly, why does Emily have this sense of belonging? Is it because of the grandeur of the Canyon,or is it something within her that we don't know about? Secondly, how does this sense of belonging translate into the narrative? Without more context, it's hard to fully evaluate the significance of this moment. However, it certainly provides a strong emotional anchor for the reader, setting up a compelling personal journey for Emily.



=================
dialogue.messages
=================

The output below shows the messages saved in dialogue.messages:
ID: 87f7bad3-510f-4bf0-abda-0f490dfd392d
  Sender: User
  Recipient: Sara
Body:
{
    "type": "chat.send",
    "dialogue_id": "00000000-0000-0000-0000-000000000000",
    "round_no": 0,
    "step_no": 0,
    "message_id": "00000000-0000-0000-0000-000000000000.1",
    "content_type": "text",
    "role": "user",
    "content": "Tell me a one paragraph story."
}

----------

ID: e4f4173e-2e66-4d53-bd27-bbccd7f53fe6
  Sender: Sara
  Recipient: User
Body:
{
    "type": "chat.reply",
    "dialogue_id": "00000000-0000-0000-0000-000000000000",
    "round_no": 0,
    "step_no": 1,
    "message_id": "00000000-0000-0000-0000-000000000000.3",
    "chunk_no": 5,
    "chunk": "<eom>",
    "content_type": "text",
    "role": "assistant",
    "content": "As she stood alone on the deserted beach, Emily felt an icy breeze whisper her name just as the clock struck midnight, exactly 20 years after her grandfather's mysterious disappearance."
}